In [74]:
import pickle
import gensim
import pyLDAvis
import pyLDAvis.gensim_models
import spacy
import pandas as pd
import nltk; nltk.download('stopwords')
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import re
import warnings
from pprint import pprint
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import seaborn as sns
%config InlineBackend.figure_formats = ['retina']
from sklearn.metrics import f1_score
from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.metrics import fbeta_score
import matplotlib.pyplot as plt

[nltk_data] Downloading package stopwords to /Users/hieu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [75]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

df = pd.read_pickle("../data/pickle/final_wo_errors.pkl")
df = df.drop(1727)

df['nlp'] = df['description_aims']
df["nlp"] = df["nlp"].fillna("")

stop_words.extend(["service", "deliver", "programme", "support", "provide", "new", "system", "government", "projects", "project"])


In [76]:
nlp = spacy.load('en_core_web_lg', disable=['parser', 'ner'])

In [77]:
# Create a dictionary of the characters to replace
replacements = {
    'Âs': ' ',
    '\xa0': ' ',
    '\n': ' ',
    '\r': ' ',
    '\x96': ' ',
    '\t': ' ',
    "\'s": ' ',
    "'s": ' ',
    '\x92': ' ',
    '\x95': ' ',
    '\x93': ' ',
    '\x94': ' ',
    '\x91': ' '
}

# Replace the unwanted characters
def clean_text(text):
    for old, new in replacements.items():
        text = text.replace(old, new)
    return text

df["nlp"] = df["nlp"].apply(clean_text)

In [78]:
def strip_newline(series):
    return [review.replace('\n','') for review in series]

df["nlp"] = strip_newline(df["nlp"])

In [79]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

words = list(sent_to_words(df["nlp"]))

In [80]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

words = remove_stopwords(words)

In [81]:
def bigrams(words, bi_min=15, tri_min=10):
    bigram = gensim.models.Phrases(words, min_count = bi_min)
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    return bigram_mod

bigram = bigrams(words)

bigram_train4 = [bigram[review] for review in words]

In [82]:
train_id2word4 = gensim.corpora.Dictionary(bigram_train4)
train_id2word4.filter_extremes(no_below=10, no_above=0.35)
train_id2word4.compactify()
train_corpus4 = [train_id2word4.doc2bow(text) for text in bigram_train4]

In [83]:
import logging
logging.basicConfig(filename='lda_model.log', format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    lda_train4 = gensim.models.ldamulticore.LdaMulticore(
                           corpus=train_corpus4,
                           num_topics=20,
                           id2word=train_id2word4,
                           chunksize=100,
                           workers=7, # Num. Processing Cores - 1
                           passes=50,
                           eval_every = 1,
                           per_word_topics=True)
    lda_train4.save('lda_train4.model')

In [84]:
coherence_model_lda = CoherenceModel(model=lda_train4, texts=bigram_train4, dictionary=train_id2word4, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
coherence_lda

0.38095500055990345

In [104]:
topic_words = lda_train4.print_topics(20,num_words=10)
type(topic_words)


list

In [86]:
train_vecs = []
for i in range(len(df)):
    top_topics = lda_train4.get_document_topics(train_corpus4[i], minimum_probability=0.0)
    topic_vec = [top_topics[i][1] for i in range(20)]
    train_vecs.append(topic_vec)

In [87]:
train_corpus4

[[(0, 1),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 1)],
 [(3, 3),
  (13, 2),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 3),
  (19, 3),
  (20, 6),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 2),
  (29, 1),
  (30, 1),
  (31, 1),
  (32, 1),
  (33, 1),
  (34, 1),
  (35, 1),
  (36, 1),
  (37, 1),
  (38, 1),
  (39, 1),
  (40, 1),
  (41, 1),
  (42, 1),
  (43, 1),
  (44, 1)],
 [(13, 1),
  (14, 1),
  (19, 1),
  (45, 1),
  (46, 1),
  (47, 1),
  (48, 1),
  (49, 1),
  (50, 4),
  (51, 1),
  (52, 1),
  (53, 1),
  (54, 1),
  (55, 2),
  (56, 1),
  (57, 2),
  (58, 1),
  (59, 3),
  (60, 1),
  (61, 1),
  (62, 1),
  (63, 1),
  (64, 1),
  (65, 1),
  (66, 1),
  (67, 1),
  (68, 1),
  (69, 1),
  (70, 2),
  (71, 1),
  (72, 1),
  (73, 1),
  (74, 1),
  (75, 1),
  (76, 1),
  (77, 1),
  (78, 1),
  (79, 1),
  (80, 1),
  (81, 1),
  (82, 1),
  (83, 2),
  (84, 1),
  (85, 1),
  (86, 1

In [88]:
train_vecs[0:2]

[[0.0033336245,
  0.0033336245,
  0.0033336245,
  0.0033336245,
  0.0033336245,
  0.0033336245,
  0.14962126,
  0.0033336245,
  0.0033336245,
  0.49102482,
  0.0033336245,
  0.0033336245,
  0.0033336245,
  0.3026823,
  0.0033336245,
  0.0033336245,
  0.0033336245,
  0.0033336245,
  0.0033336245,
  0.0033336245],
 [0.0010638819,
  0.0010638819,
  0.0010638819,
  0.0010638819,
  0.0010638819,
  0.0010638819,
  0.0010638819,
  0.0010638819,
  0.0010638819,
  0.0010638819,
  0.0010638819,
  0.0010638819,
  0.0010638819,
  0.0010638819,
  0.0010638819,
  0.0010638819,
  0.0010638819,
  0.0010638819,
  0.0010638819,
  0.9797862]]

In [90]:
new_doc = "The Smart City Infrastructure Modernization Project aims to enhance the quality of public services, improve urban livability, and drive sustainable development through the integration of smart technologies in city infrastructure. The project will leverage advanced technologies like the Internet of Things (IoT), AI-driven analytics, and green energy solutions to transform the city’s transportation, energy, public safety, and environmental systems. Improve urban mobility: Implement smart traffic management systems, real-time public transport tracking, and EV charging infrastructure to reduce congestion and pollution. Enhance public safety: Deploy intelligent surveillance and emergency response systems for faster and more effective public safety interventions. Sustainable energy solutions: Integrate smart energy grids and renewable energy sources to optimize energy use, reduce carbon emissions, and ensure uninterrupted power supply to critical infrastructure. Citizen engagement: Develop a user-friendly platform that allows citizens to report issues, receive real-time updates on public services, and actively participate in decision-making processes. Environmental monitoring: Utilize IoT sensors to monitor air and water quality, waste management, and other environmental factors to ensure sustainability and regulatory compliance. Aims: Reduced traffic congestion by 20% through smart traffic management. A 15% reduction in energy consumption in public buildings by integrating smart energy solutions. Improved citizen satisfaction by 25% due to real-time access to city services and improved communication. Enhanced public safety with a 10% reduction in emergency response times."

In [105]:
# Create a DataFrame (assuming you have a 'project_name' column as well)
df["topic_distribution"] =  pd.DataFrame({'topic_distribution': train_vecs})
df['topic'] = df['topic_distribution'].apply(lambda x: x.index(max(x)))
df['topic_words'] = df['topic'].apply(lambda x: topic_words[x][1])

In [108]:
df.to_pickle("../data/pickle/data_with_topics.pkl")